In [2]:
import os
from typing import List, Dict, Any
import streamlit as st
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, AIMessage, SystemMessage
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.document_loaders import TextLoader, PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate
import tempfile

In [3]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import HuggingFacePipeline
from langchain.chains import RetrievalQA
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import TextLoader

from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
import torch


/Users/aadityaraj/Projects/GenAI/langchain_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
from langchain.llms import Ollama

def run_rag(query:str)->str:

    # Step 1: Load local knowledge base
    loader = TextLoader("data/knowledge.txt")  # Replace with your own file
    documents = loader.load()

    # Step 2: Split documents into chunks
    text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    docs = text_splitter.split_documents(documents)

    # Step 3: Use local embedding model (MiniLM - free & fast)
    embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
    vectorstore = FAISS.from_documents(docs, embedding_model)

    # Step 4: Use local LLM via Ollama (e.g., mistral, llama3, gemma)
    llm = Ollama(model="mistral")  # You can change to "llama3", "gemma", etc.

    # Step 5: Create the RetrievalQA chain
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        retriever=vectorstore.as_retriever(),
        chain_type="stuff"
    )

    ####### For Backend and Test Uses ########
    # Step 6: Interactive chatbot loop
    print("🤖 RAG Chatbot (Ollama-based) is ready! Type your question or 'exit'.")
    while True:
        query = input("You: ")
        if query.lower() == "exit":
            break
        answer = qa_chain.run(query)
        print(f"Bot: {answer}")

In [16]:
# !ollama pull mistral